In [1]:
import os, subprocess
import json
import uproot3
import awkward as ak
import numpy as np
from coffea import processor, util, hist

from plotter import *

In [2]:
lumis = {}
lumis['2016'] = 35.9
lumis['2017'] = 41.5
lumis['2018'] = 59.9

with open('xsec.json') as f:
  xs = json.load(f)

with open('pmap.json') as f:
  pmap = json.load(f)

systematics = ['nominal',
               'jet_triggerUp','jet_triggerDown',
               'btagWeightUp','btagWeightDown','btagEffStatUp','btagEffStatDown',
               'UESUp','UESDown','JESUp','JESDown','JERUp','JERDown',
              ]

In [3]:
year = '2016'
nfiles = len(subprocess.getoutput("ls infiles-split/"+year+"*.json").split())
outsum = processor.dict_accumulator()

In [ ]:
# Load all files
for n in range(1,nfiles+1):

    with open('infiles-split/'+year+'_'+str(n)+'.json') as f:
      infiles = json.load(f)
    
    filename = '/myeosdir/vbf-category/outfiles/'+year+'_'+str(n)+'.coffea'
    if os.path.isfile(filename):
        out = util.load(filename)
        outsum.add(out)
    else:
        print(n,infiles.keys())
        #print("File " + filename + " is missing")
        
scale_lumi = {k: xs[k] * 1000 *lumis[year] / w for k, w in outsum['sumw'].items()}
outsum['templates'].scale(scale_lumi, 'dataset')
outsum['templates-vbf'].scale(scale_lumi, 'dataset')
outsum['muonkin'].scale(scale_lumi, 'dataset')
outsum['mujetkin'].scale(scale_lumi, 'dataset')
outsum['cutflow_msd'].scale(scale_lumi, 'dataset')

1 dict_keys(['DYJetsToLL_M-50_HT-1200to2500_TuneCUETP8M1_13TeV-madgraphMLM-pythia8'])
3 dict_keys(['DYJetsToLL_M-50_HT-2500toInf_TuneCUETP8M1_13TeV-madgraphMLM-pythia8'])
5 dict_keys(['DYJetsToLL_M-50_HT-400to600_TuneCUETP8M1_13TeV-madgraphMLM-pythia8'])
6 dict_keys(['DYJetsToLL_M-50_HT-400to600_TuneCUETP8M1_13TeV-madgraphMLM-pythia8'])
7 dict_keys(['DYJetsToLL_M-50_HT-400to600_TuneCUETP8M1_13TeV-madgraphMLM-pythia8'])
8 dict_keys(['DYJetsToLL_M-50_HT-400to600_TuneCUETP8M1_13TeV-madgraphMLM-pythia8'])
9 dict_keys(['DYJetsToLL_M-50_HT-400to600_TuneCUETP8M1_13TeV-madgraphMLM-pythia8'])
10 dict_keys(['DYJetsToLL_M-50_HT-400to600_TuneCUETP8M1_13TeV-madgraphMLM-pythia8'])
11 dict_keys(['DYJetsToLL_M-50_HT-400to600_TuneCUETP8M1_13TeV-madgraphMLM-pythia8'])
13 dict_keys(['DYJetsToLL_M-50_HT-600to800_TuneCUETP8M1_13TeV-madgraphMLM-pythia8'])
14 dict_keys(['DYJetsToLL_M-50_HT-600to800_TuneCUETP8M1_13TeV-madgraphMLM-pythia8'])
15 dict_keys(['DYJetsToLL_M-50_HT-600to800_TuneCUETP8M1_13TeV-madgrap

In [ ]:
cutflow = outsum['cutflow_msd'].group('dataset', hist.Cat('process', 'Process'), pmap).sum('genflavor').integrate('region','muoncontrol').sum('msd')

In [ ]:
cutflow.integrate('process','muondata').values()

In [ ]:
cutflow.integrate('process','ttbar').values()

In [ ]:
templates = outsum['templates'].group('dataset', hist.Cat('process', 'Process'), pmap).integrate('systematic','nominal')
muonkin = outsum['muonkin'].group('dataset', hist.Cat('process', 'Process'), pmap).integrate('systematic','nominal')
mujetkin = outsum['mujetkin'].group('dataset', hist.Cat('process', 'Process'), pmap).integrate('systematic','nominal')

In [ ]:
h = templates.sum('ddb1','pt1').integrate('region', 'muoncontrol')
plot_datamc(h,year+'/inclusive/muCR_msd1',year+" muon CR")

In [ ]:
h = mujetkin.integrate('region', 'muoncontrol').sum('msd1','eta1','ddb1')
plot_datamc(h,year+'/inclusive/muCR_pt1',year+" muon CR")

In [ ]:
h = mujetkin.integrate('region', 'muoncontrol').sum('msd1','pt1','ddb1')
plot_datamc(h,year+'/inclusive/muCR_eta1',year+" muon CR")

In [ ]:
h = mujetkin.integrate('region', 'muoncontrol').sum('msd1','pt1','eta1')
plot_datamc(h,year+'/inclusive/muCR_ddb1',year+" muon CR")

In [ ]:
h = muonkin.integrate('region', 'muoncontrol').sum('ptmu','etamu').integrate('ddb1',int_range=slice(0.89,1))
plot_datamc(h,year+'/inclusive/muCR_msd1_pass',year+" muon CR, DBB pass")
h = muonkin.integrate('region', 'muoncontrol').sum('ptmu','etamu').integrate('ddb1',int_range=slice(0,0.89))
plot_datamc(h,year+'/inclusive/muCR_msd1_fail',year+" muon CR, DDB fail")
h = muonkin.integrate('region', 'muoncontrol').sum('ptmu','etamu','ddb1')
plot_datamc(h,year+'/inclusive/muCR_msd1',year+" muon CR")  

In [ ]:
h = muonkin.integrate('region', 'muoncontrol').sum('msd1','etamu').integrate('ddb1',int_range=slice(0.89,1))
plot_datamc(h,year+'/inclusive/muCR_ptmu_pass',year+" muon CR, DDB pass")
h = muonkin.integrate('region', 'muoncontrol').sum('msd1','etamu').integrate('ddb1',int_range=slice(0,0.89))
plot_datamc(h,year+'/inclusive/muCR_ptmu_fail',year+" muon CR, DDB fail")
h = muonkin.integrate('region', 'muoncontrol').sum('msd1','etamu','ddb1')
plot_datamc(h,year+'/inclusive/muCR_ptmu',year+" muon CR")

In [ ]:
h = muonkin.integrate('region', 'muoncontrol').sum('msd1','ptmu').integrate('ddb1',int_range=slice(0.89,1))
plot_datamc(h,year+'/inclusive/muCR_etamu_pass',year+" muon CR, DDB pass")
h = muonkin.integrate('region', 'muoncontrol').sum('msd1','ptmu').integrate('ddb1',int_range=slice(0,0.89))
plot_datamc(h,year+'/inclusive/muCR_etamu_fail',year+" muon CR, DDB fail")
h = muonkin.integrate('region', 'muoncontrol').sum('msd1','ptmu','ddb1')
plot_datamc(h,year+'/inclusive/muCR_etamu',year+" muon CR")